In [10]:
import numpy as np
import geopandas as gp
import multiprocessing as mp
import libpysal as ps
import sys
# sys.path.append('/Users/Ziqi/Desktop/mgwr/')
from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW

In [11]:
import mgwr

In [12]:
#Load Berlin example
prenz = gp.read_file(ps.examples.get_path('prenzlauer.zip'))

In [13]:
b_y = np.log(prenz['price'].values.reshape((-1, 1)))
b_X = prenz[['review_sco','accommodat','bathrooms']].values 
b_X = (b_X - b_X.mean(axis=0)) / b_X.std(axis=0)
b_y = (b_y - b_y.mean(axis=0)) / b_y.std(axis=0)
u = prenz['X']
v = prenz['Y']
b_coords = list(zip(u, v))

In [14]:
#This might be needed to turn off the OpenMP multi-threading
%env OMP_NUM_THREADS = 1

env: OMP_NUM_THREADS=1


### GWR No Parallel

In [15]:
%%time
gwr_selector = Sel_BW(b_coords, b_y, b_X)
gwr_bw = gwr_selector.search()
print(gwr_bw)
gwr_results = GWR(b_coords, b_y, b_X, gwr_bw).fit()

192.0
CPU times: user 2min 46s, sys: 9min 24s, total: 12min 11s
Wall time: 46.8 s


### MGWR No Parallel

In [16]:
%%time
mgwr_selector = Sel_BW(b_coords, b_y, b_X, multi=True)
mgwr_bw = mgwr_selector.search()
print(mgwr_bw)
mgwr_results = MGWR(b_coords, b_y, b_X, selector=mgwr_selector).fit()

Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

[ 191. 1279.   79. 2200.]


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 31min 1s, sys: 1h 41min 50s, total: 2h 12min 52s
Wall time: 8min 21s


In [17]:
#Parrallelization is more favored when you your data are large and/or your machine have many many cores.
#mgwr has soft dependency of numba, please install numba if you need better performance (pip install numba).

n_proc = 2 #two processors
pool = mp.Pool(n_proc) 

### GWR Parallel

In [18]:
%%time
gwr_selector = Sel_BW(b_coords, b_y, b_X)
gwr_bw = gwr_selector.search(pool=pool) #add pool to Sel_BW.search
print(gwr_bw)
gwr_results = GWR(b_coords, b_y, b_X, gwr_bw).fit(pool=pool) #add pool to GWR.fit

192.0
CPU times: user 1.51 s, sys: 0 ns, total: 1.51 s
Wall time: 30.1 s


### MGWR Parallel

In [19]:
%%time
mgwr_selector = Sel_BW(b_coords, b_y, b_X, multi=True)
mgwr_bw = mgwr_selector.search(pool=pool) #add pool to Sel_BW.search
print(mgwr_bw)
mgwr_results = MGWR(b_coords, b_y, b_X, selector=mgwr_selector).fit(pool=pool) #add pool to MGWR.fit

Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

[ 191. 1279.   79. 2200.]


Inference:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 29.2 s, sys: 0 ns, total: 29.2 s
Wall time: 6min 41s


In [20]:
pool.close() # Close the pool when you finish
pool.join()